In [2]:
from datetime import datetime, timezone
from datetime import datetime
from tqdm.notebook import tqdm
import pandas as pd
from gspread_dataframe import get_as_dataframe
import gspread
import os
import shutil
import subprocess
import re
import time

# ---------------------------
# Directories
# ---------------------------
INPUT_DIR = "Downloaded_Universities"
PROCESSED_DIR = os.path.join(INPUT_DIR, "Processed_Universities")
os.makedirs(PROCESSED_DIR, exist_ok=True)

# ---------------------------
# Google Sheet Config
# ---------------------------
SERVICE_ACCOUNT_FILE = "service_account.json"
SHEET_ID = "1eYz8Nvr3BToRrmReXNLR8zQrk4X8tsdKZO_Fj9mNThc"
SHEET_NAME = "Scrapper Running"

# ---------------------------
# Helper functions
# ---------------------------


LOG_DIR = "logs"
os.makedirs(LOG_DIR, exist_ok=True)


def log_time(csv_file, elapsed_seconds):
    """Append elapsed time info for a university to a daily log file in H:M:S format."""
    uni_name = os.path.splitext(csv_file)[0]
    ts = datetime.now(timezone.utc).strftime("%Y-%m-%d %H:%M:%S")

    hours, remainder = divmod(elapsed_seconds, 3600)
    minutes, seconds = divmod(remainder, 60)
    elapsed_str = f"{int(hours)}h {int(minutes)}m {int(seconds)}s"

    log_file = os.path.join(
        LOG_DIR, f"elapsed_{datetime.now(timezone.utc).strftime('%Y-%m-%d')}.log")

    with open(log_file, "a", encoding="utf-8") as f:
        f.write(f"[{ts}] {uni_name}: {elapsed_str}\n")


def safe_filename(name: str) -> str:
    """Clean a string to make it safe for file names."""
    if not name:
        return ""
    return re.sub(r'[.\'<>:"/\\|?*]', '', name).strip().lower()


def update_sheet_status(csv_file_name, status):
    """Update 'Scraping?' column for matching university CSV in the sheet."""
    uni_name_from_csv = safe_filename(os.path.splitext(csv_file_name)[0])

    for i, uni_name in df_sheet['University Name'].items():
        if uni_name and safe_filename(str(uni_name)) == uni_name_from_csv:
            df_sheet.at[i, 'Scraping?'] = status


# ---------------------------
# Authenticate Google Sheet
# ---------------------------
gc = gspread.service_account(filename=SERVICE_ACCOUNT_FILE)
sh = gc.open_by_key(SHEET_ID)
ws = sh.worksheet(SHEET_NAME)
df_sheet = get_as_dataframe(ws, evaluate_formulas=True, header=0)

# Ensure 'Scraping?' column exists
if 'Scraping?' not in df_sheet.columns:
    df_sheet['Scraping?'] = "Pending"

# ---------------------------
# Spider runner
# ---------------------------


def run_spider_on_csvs():
    csv_files = [f for f in os.listdir(
        INPUT_DIR) if f.lower().endswith(".csv")]

    # Create a single notebook tqdm bar
    pbar = tqdm(total=len(csv_files), desc="Processing CSVs")

    for csv_file in csv_files:
        csv_path = os.path.join(INPUT_DIR, csv_file)
        tqdm.write(f"\n🚀 Running spider for: {csv_file}")
        start_time = time.time()

        # Mark as processing
        update_sheet_status(csv_file, "Processing")
        ws.update([df_sheet.columns.values.tolist()] +
                  df_sheet.fillna("").values.tolist())

        try:
            subprocess.run([
                "python", "-m", "scrapy", "crawl", "courses",
                f"-a", f"csv_file={csv_path}"
            ], check=True)

            # Move processed CSV
            shutil.move(csv_path, os.path.join(PROCESSED_DIR, csv_file))
            print(f"✅ Finished and moved: {csv_file}")

            # Count number of .txt files in the output folder (same as CSV name)
            output_folder = os.path.splitext(csv_file)[0]
            txt_files_count = 0
            folder_path = os.path.join(output_folder)
            if os.path.exists(folder_path):
                txt_files_count = len([f for f in os.listdir(
                    folder_path) if f.lower().endswith(".txt")])

            # Update Google Sheet 'Scraped Count' column
            uni_name_from_csv = safe_filename(os.path.splitext(csv_file)[0])
            for i, uni_name in df_sheet['University Name'].items():
                if uni_name and safe_filename(str(uni_name)) == uni_name_from_csv:
                    df_sheet.at[i, 'Scraped Count'] = txt_files_count
                    df_sheet.at[i, 'Scraping?'] = "Processed"

        except subprocess.CalledProcessError as e:
            print(f"❌ Spider failed for {csv_file}: {e}")
            update_sheet_status(csv_file, "Failed")
            txt_files_count = 0

        elapsed = time.time() - start_time
        tqdm.write(f"⏱️ Time elapsed for {csv_file}: {elapsed:.2f} seconds")

        # log to file
        log_time(csv_file, elapsed)

        # Update the same tqdm bar
        pbar.update(1)

    pbar.close()

    # Push final status to Google Sheet including Scraped Count
    ws.update([df_sheet.columns.values.tolist()] +
              df_sheet.fillna("").values.tolist())
    print("✅ Sheet updated with Scraping? and Scraped Count statuses")


# ---------------------------
# Run all CSVs
# ---------------------------
if __name__ == "__main__":
    run_spider_on_csvs()

KeyboardInterrupt: 